In [1]:
DATA_NAME = 'pastis-full' 
TRANSFORM = 'wavelet-diagonal'
CHANNEL = 'green'

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_green_diagonal_wavelet_full_pastis.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
        cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,2,3,4,5,6,7,8
2,6930.15,0.01,0.09,0.01,0.01,-0.00,0.00
3,0.01,1807.62,-0.01,-0.01,-0.00,0.00,0.00
4,0.09,-0.01,377.31,-0.00,-0.00,-0.00,-0.00
5,0.01,-0.01,-0.00,58.96,-0.00,0.00,-0.00
6,0.01,-0.00,-0.00,-0.00,8.04,0.00,0.00
7,-0.00,0.00,-0.00,0.00,0.00,0.94,0.00
8,0.00,0.00,-0.00,-0.00,0.00,0.00,0.05


In [10]:
corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))
corr_df = pd.DataFrame(corr_matrix, index=GROUPS, columns=GROUPS)
corr_df.round(5)

,2,3,4,5,6,7,8
2,1.00000,0.00000,0.00005,0.00001,0.00003,-0.00002,0.00002
3,0.00000,1.00000,-0.00001,-0.00004,-0.00003,0.00000,0.00001
4,0.00005,-0.00001,1.00000,-0.00003,-0.00002,-0.00002,-0.00000
5,0.00001,-0.00004,-0.00003,1.00000,-0.00002,0.00000,-0.00001
6,0.00003,-0.00003,-0.00002,-0.00002,1.00000,0.00002,0.00001
7,-0.00002,0.00000,-0.00002,0.00000,0.00002,1.00000,0.00004
8,0.00002,0.00001,-0.00000,-0.00001,0.00001,0.00004,1.00000


In [11]:
np.linalg.norm(cov_matrix - np.diag(np.diag(cov_matrix)))  # Remove diagonal elements for visualization

0.1301331686208936

In [12]:
pca = PCA()
pca.fit(cov_matrix)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[6.87539955e+06 4.53766064e+05 1.89755458e+04 4.34801825e+02
 7.18507545e+00 7.36600755e-02 7.13367143e-35]

Principal components (eigenvectors):


,0,1,2,3,4,5,6
0,0.998879,-0.046463,-0.008948,-0.001394,-0.000189,-0.000023,-0.000001
1,0.046017,0.997950,-0.043997,-0.006531,-0.000889,-0.000104,-0.000006
2,0.010895,0.043236,0.998178,-0.040304,-0.005330,-0.000624,-0.000035
3,0.002118,0.008141,0.039673,0.998085,-0.046406,-0.005297,-0.000295
4,0.000383,0.001476,0.007066,0.045753,0.997159,-0.059318,-0.003231
5,0.000068,0.000259,0.001244,0.007952,0.058735,0.996681,-0.055795
6,0.000007,0.000029,0.000139,0.000886,0.006495,0.055504,0.998437


In [13]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(len(GROUPS)), metric='cosine'))
cos_dist


array([0.00112107, 0.00205045, 0.00182228, 0.00191503, 0.00284139,
       0.00331937, 0.00156304])